# Overview

This notebook will explore the `pybaseball` package and see if it can be used to create features for a regression model.

In [109]:
from pybaseball import batting_stats_range
import datetime
import pandas as pd

### Loading stats accumulated over a date range

In [154]:
data = batting_stats_range('2017-06-11', '2017-06-11')
data.head()

,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,...,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS
1,Jose Abreu,30,340,MLB-AL,Chicago,1,4,3,1,0,...,1,0,0,2,0,0,0.000,0.250,0.000,0.250
2,Matt Adams,28,340,MLB-NL,Atlanta,1,4,4,0,0,...,0,0,0,1,0,0,0.000,0.000,0.000,0.000
3,Tim Adleman,29,340,MLB-NL,Cincinnati,1,3,3,0,1,...,0,0,0,0,0,0,0.333,0.333,0.333,0.667
4,Ehire Adrianza,27,340,MLB-AL,Minnesota,1,1,0,0,0,...,0,0,0,0,0,0,NaN,1.000,NaN,NaN
5,Jesus Aguilar,27,340,MLB-NL,Milwaukee,1,1,0,0,0,...,0,0,0,0,0,0,NaN,1.000,NaN,NaN


### Calculating FD points for a given range

Create a function that will take a single row and output the Fanduel score for that player.

In [111]:
def get_fantasy_points(stats):
    singles = stats["H"]-stats["2B"]-stats["3B"]-stats["HR"]
    points = 3*singles+3.2*stats["R"]+6*stats["2B"]+9*stats["3B"]+12*stats["HR"]\
            +3.5*stats["RBI"]+3*stats["BB"]+3*stats["IBB"]+3*stats["HBP"]+6*stats["SB"]
    return pd.DataFrame({"Name": stats["Name"],
                         "FD Points": points})

In [153]:
data.set_index("Name").join(get_fantasy_points(data).set_index("Name")).head()

,Age,#days,Lev,Tm,G,PA,AB,R,H,2B,...,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,FD Points
Name,,,,,,,,,,,,,,,,,,,,,
A.J. Ellis,36,342,MLB-NL,Miami,2,9,7,1,3,0,...,0,0,0,0,0,0.429,0.556,0.429,0.984,21.7
Aaron Altherr,26,341,MLB-NL,Philadelphia,6,25,24,2,7,2,...,0,0,1,0,0,0.292,0.320,0.583,0.903,65.4
Aaron Hicks,27,341,MLB-AL,New York,6,27,24,6,7,1,...,0,0,0,0,0,0.292,0.370,0.583,0.954,80.7
Aaron Hill,35,341,MLB-NL,San Francisco,7,12,11,1,3,2,...,0,0,0,0,0,0.273,0.333,0.636,0.970,37.7
Aaron Judge,25,341,MLB-AL,New York,6,29,24,7,9,2,...,0,0,0,1,0,0.375,0.483,0.583,1.066,98.9


### Easily getting stats within a range

Rather than passing strings to the argument of `batting_stats_range()`, a wrapper function will be written that takes a `datetime` and a `timedelta` and gets the stats accumulated within that duration.

In [117]:
cur_day = datetime.date(2017, 6, 11)
day = datetime.timedelta(days=1)
week = datetime.timedelta(days=7)
start = cur_day-week
print(cur_day)
print(start)

2017-06-11
2017-06-04


In [136]:
def get_batting_stats(cur_day, duration, forecasting=False):
    yesterday = cur_day-datetime.timedelta(days=1)
    if forecasting:
        offset = datetime.timedelta(0)
    else:
        offset = datetime.timedelta(1)
        
    results = batting_stats_range((cur_day-duration).strftime("%Y-%m-%d"),
                                  (cur_day-offset).strftime("%Y-%m-%d")
                                 )
    results = results.drop(columns=["Age", "#days", "Lev", "Tm"])
    
    if not forecasting:
        new_names = [(i,str(duration.days)+"-"+i) for i in results.iloc[:, 1:].columns.values]
        results = results.rename(columns = dict(new_names))
    
    return results

In [22]:
week_df = get_batting_stats(end, week)
day_df = get_batting_stats(end, day)

In [138]:
def construct_multi_duration_dataframe(cur_day, durations):
    dfs = [get_batting_stats(cur_day, duration, forecasting=False).set_index("Name") for duration in durations]
    
    return dfs[0].join(dfs[1:])

In [139]:
results = construct_multi_duration_dataframe(cur_day, [day, week])

In [140]:
results.head()

,1-G,1-PA,1-AB,1-R,1-H,1-2B,1-3B,1-HR,1-RBI,1-BB,...,7-HBP,7-SH,7-SF,7-GDP,7-SB,7-CS,7-BA,7-OBP,7-SLG,7-OPS
Name,,,,,,,,,,,,,,,,,,,,,
A.J. Ellis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0.429,0.556,0.429,0.984
Aaron Altherr,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0.292,0.320,0.583,0.903
Aaron Hicks,1.0,5.0,4.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0.292,0.370,0.583,0.954
Aaron Hill,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.273,0.333,0.636,0.970
Aaron Judge,1.0,5.0,4.0,3.0,3.0,1.0,0.0,1.0,3.0,1.0,...,0,0,0,0,1,0,0.375,0.483,0.583,1.066


In [141]:
def add_fantasy_points(cur_day, dataframe):
    points = get_fantasy_points(get_batting_stats(cur_day,
                                                  datetime.timedelta(0), 
                                                  forecasting=True)).set_index("Name")
    return dataframe.join(points)

In [142]:
def remove_nan_fd_points(dataframe):
    return dataframe[dataframe["FD Points"]>= 0]

In [143]:
new_results = add_fantasy_points(cur_day, results)
new_results = remove_nan_fd_points(new_results)

In [144]:
new_results.head()

,1-G,1-PA,1-AB,1-R,1-H,1-2B,1-3B,1-HR,1-RBI,1-BB,...,7-SH,7-SF,7-GDP,7-SB,7-CS,7-BA,7-OBP,7-SLG,7-OPS,FD Points
Name,,,,,,,,,,,,,,,,,,,,,
Aaron Altherr,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0.292,0.320,0.583,0.903,0.0
Aaron Hicks,1.0,5.0,4.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0.292,0.370,0.583,0.954,25.4
Aaron Hill,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.273,0.333,0.636,0.970,9.7
Aaron Judge,1.0,5.0,4.0,3.0,3.0,1.0,0.0,1.0,3.0,1.0,...,0,0,0,1,0,0.375,0.483,0.583,1.066,59.3
Aaron Nola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0.000,0.000,0.000,0.000,0.0
